In [ ]:
nltk.download('stopwords')

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import re
import string
import sys
import os

import jellyfish # library for fuzzy text comparisons
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

from tqdm import tqdm_pandas, tqdm_notebook as tqdm
from tqdm import tqdm as tqorig
tqorig.pandas(tqdm)
from autocorrect import spell
from textblob import TextBlob

[nltk_data] Downloading package wordnet to /Users/natali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
client = MongoClient()
db     = client.fletcher
reviews   = db.reviews

In [4]:
cursor = reviews.find({},{ 'text_review OVERALL UNIVERSITY RATING': 1, 'college' : 1})

In [5]:
df_overall_review = pd.DataFrame(list(cursor))

In [6]:
text = list(df_overall_review[df_overall_review['college'] == 'University of Cumbria']['text_review OVERALL UNIVERSITY RATING'])

In [99]:
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
letters_only = lambda x: re.sub("[^a-zA-Z]", " ", str(x))


df_overall_review['reviews'] = df_overall_review['text_review OVERALL UNIVERSITY RATING'].map(letters_only).map(alphanumeric).map(punc_lower)
df_overall_review.head()

,_id,college,text_review OVERALL UNIVERSITY RATING,reviews,polarity
0,5c75db20c0bfe17384b4d259,"Northumbria University, Newcastle",great and helpful experience\nprovide good tea...,great and helpful experience provide good teac...,0.687500
1,5c75db20c0bfe17384b4d262,University of Cumbria,enjoying it. good mix of lectures and practica...,enjoying it good mix of lectures and practica...,0.268056
2,5c75db20c0bfe17384b4d26b,"Northumbria University, Newcastle","Sports centre is really a amazing place, Infra...",sports centre is really a amazing place infra...,0.400000
3,5c75db20c0bfe17384b4d274,University of Hull,university has been interesting the first year...,university has been interesting the first year...,-0.035000
4,5c75db20c0bfe17384b4d27d,"Northumbria University, Newcastle","I am quite enjoy my university life at here, t...",i am quite enjoy my university life at here t...,0.033333


In [98]:
df_overall_review['polarity'] = df_overall_review['reviews'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [8]:
X = df_overall_review.reviews

In [9]:
stemmer = SnowballStemmer('english')
new_corpus=[' '.join([stemmer.stem(word) for word in text.split(' ')])
          for text in X]

In [10]:
lemmer=WordNetLemmatizer()
new_corpus = [' '.join([lemmer.lemmatize(word) for word in text.split(' ')])
          for text in new_corpus]

In [13]:
cv1 = CountVectorizer(stop_words = 'english')

X_cv1 = cv1.fit_transform(new_corpus)

#sm = pd.DataFrame(X_cv1.toarray(), columns=cv1.get_feature_names()).head()

In [ ]:
words = cv1.get_feature_names()

In [ ]:
words[9]

In [95]:
blob = TextBlob(new_corpus[78])

In [96]:
blob

TextBlob("overal  it been  ok  no where near a amaz a what i expect  it s basic you just go listen to lectur and go to lab  easi stuff  then i got stuck and got no help  so far  on the academ side it been a bit rubbish  howev  on the societi side  im have a great time  probabl spend more time in the radio station than the librari but it great  so overal  uni isnt amaz but not horribl  it s just ok ")

In [97]:
blob.sentiment.polarity

0.4125

In [65]:
blob = TextBlob(new_corpus[5])

In [ ]:
spell(a[2].split(' ')[1])

In [ ]:
spell('suitabl cours')

In [ ]:
spell(cv1.get_feature_names()[9])

In [ ]:
def get_all_glove(glove_path):
    # create word_vec with glove vectors
    glove_dict = {}
    with open(glove_path) as f:
        for line in tqdm(f):
            word, vec = line.split(' ', 1)
            glove_dict[word] = np.array(list(map(float, vec.split())), dtype=np.float32)
    return glove_dict

In [ ]:
PATH_TO_GLOVE = '/Users/natali/Downloads/glove/glove.6B.50d.txt'

In [ ]:
glove = get_all_glove(PATH_TO_GLOVE)

In [ ]:
def getNeighbours(word_list, transform_vector=0, c=1.0, neighbours=10, metric='cosine', use_faiss=FAISS_AVAILABLE):
    word_embeds = np.vstack([emb[w2id[x]] for x in word_list]) # create a numpy array of word embeddings

    if use_faiss:
        distances, indices = index.search(
            (word_embeds - transform_vector*c).astype(np.float32), neighbours)
    else:
        dist_matrix = cdist(
            (word_embeds - transform_vector*c).astype(np.float32), emb, metric=metric)
        indices = np.argsort(dist_matrix)[:, :neighbours]
    return indices

In [104]:
tweets = [item for item in text] # all tweets

In [107]:
import nltk
from   nltk.stem.snowball import SnowballStemmer
from   more_itertools import unique_everseen

stopwords = nltk.corpus.stopwords.words('english')
stopwords.append(u'\u0027s')
stopwords.append(u'n\u0027t')
stopwords.append(u'\u0027m')
stopwords.append("'re")
stopwords.append("'ve")
print('Stop words (%i): \n%s\n' %(len(stopwords),[str(word) for word in stopwords]))

stemmer           = SnowballStemmer("english")
stemmed_stopwords = list(unique_everseen([stemmer.stem(word) for word in stopwords]))
print('Stemmed stop words (%i): \n%s\n' %(len(stemmed_stopwords),[str(word) for word in stemmed_stopwords]))

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as its own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

# Process the tweets:
tokenized_tweets = [tokenize_only(tweet) for tweet in tweets]
stemmed_tweets   = [[stemmer.stem(word) for word in tweet] for tweet in tokenized_tweets]
filtered_tweets  = [[word for word in tweet if word not in stemmed_stopwords] for tweet in stemmed_tweets]

print('Example tweet after tokenization, stemming, and filtering:')
ind = 0
print(tokenized_tweets[ind])
print(stemmed_tweets[ind])
print(filtered_tweets[ind])
print(' ')

# Create a vocabulary to convert stems back to words:
vocab = {}
assert len(tokenized_tweets) == len(stemmed_tweets), 'Tokenized tweets: %i, Stemmed tweets: %i'\
                                                     %(len(tokenized_tweets),len(stemmed_tweets))
for tokenized_tweet,stemmed_tweet in zip(tokenized_tweets,stemmed_tweets):
    assert len(tokenized_tweet) == len(stemmed_tweet), \
            'Tweet length error:\n   tokenized tweet: "%s"\n   stemmed tweet: "%s"' %(tokenized_tweet,stemmed_tweet)
    for word,stem in zip(tokenized_tweet,stemmed_tweet):
        if stem not in stemmed_stopwords:
            vocab.setdefault(stem,word)
        
print('Length of vocabulary = %i' %len(vocab))
print('Vocabulary Extract:\n %s' %([': '.join((str(key),value)) for (key,value) in vocab.items()[-50:]]))

Stop words (184): 
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'sam

TypeError: expected string or bytes-like object